In [4]:
import os
from tqdm.notebook import tqdm

In [4]:
SPLIT = 'val' # 'train' or 'val'
LIST_DIR = 'data/list'
DATASET_DIR = 'data/sets/TMGN_1814'
DATASET_DIR_MAP = f'{DATASET_DIR}/{SPLIT}/map_256'
DATASET_DIR_SAT = f'{DATASET_DIR}/{SPLIT}/rs_256'

TARGET_ZOOM_MIN = 14  # 你可以根据需要修改这个值

In [5]:
def exist_or_create(path):
    if not os.path.exists(path):
        os.makedirs(path)

def clean_up_side_image(dir_path1, dir_path2):
    target_set = set(os.listdir(dir_path1))
    for img_name in os.listdir(dir_path2):
        if img_name in target_set:
            continue
        else:
            os.remove(os.path.join(dir_path2, img_name))

In [ ]:
list_filenames_map = os.listdir(DATASET_DIR_MAP)
list_filenames_sat = os.listdir(DATASET_DIR_SAT)

print("map:",len(list_filenames_map))
print("sat:",len(list_filenames_sat))

In [ ]:
import cv2

INPUT_DIR = f'{DATASET_DIR}/{SPLIT}/map_512'
OUTPUT_DIR = f'{DATASET_DIR}/{SPLIT}/map_256_cv2'
exist_or_create(OUTPUT_DIR)

for filename in tqdm(list_filenames_map):
    # 检查文件是否为 PNG 图像
    if filename.endswith('.png'):
        # 读取图像
        parent_img = cv2.imread(os.path.join(DATASET_DIR_MAP, filename), cv2.IMREAD_UNCHANGED)

        # 检查图像是否为 512 大小
        if parent_img.shape[0] == 512 and parent_img.shape[1] == 512:
            # 转换图像大小为 256
            resized_img = cv2.resize(parent_img, (256, 256), interpolation = cv2.INTER_AREA)

            # 保存转换后的图像
            cv2.imwrite(os.path.join(OUTPUT_DIR, filename), resized_img)

In [ ]:
import shutil
import re

def process_filename(filename):
    # 使用正则表达式匹配文件名中的特定部分
    match = re.match(r"(\d+_\d+_\d+)_.*\.png", filename)
    if match:
        # 如果匹配成功，取出匹配的部分
        new_filename = match.group(1)
        # 拼接成新的文件名
        new_filename += ".png"
        return new_filename

dir_output = 'data/val/refmap-level-val-s4c-1816/val/class'
dir_image = "data/val/refmap-level-val-s4c-1816/val/step--0"
list_filenames = os.listdir(dir_image)

for filename in tqdm(list_filenames):
    filename_clean = process_filename(filename)
    # 检查文件是否为 PNG 图像
    if filename.endswith('.png'):
        # 读取图像
        if filename.endswith('samples.png'):
            dir_copy = os.path.join(dir_image, 'samples')
        elif filename.endswith('source.png'):
            dir_copy = os.path.join(dir_image, 'source')
        elif filename.endswith('target.png'):
            dir_copy = os.path.join(dir_image, 'target')
        elif filename.endswith('ref_scale.png'):
            dir_copy = os.path.join(dir_image, 'ref_scale')

        if not os.path.exists(dir_copy):
            os.makedirs(dir_copy)

        shutil.copy(os.path.join(dir_image, filename), os.path.join(dir_copy, filename_clean))

In [ ]:
dir_output = 'data/val/refmap-level-val-s4c-1816/val/level'
dir_image = "data/val/refmap-level-val-s4c-1816/val/class"

list_dirs = os.listdir(dir_image)
for dir_name in list_dirs:
    print(dir_name)
    dir_class = os.path.join(dir_image, dir_name)
    list_filenames = os.listdir(dir_class)
    for filename in tqdm(list_filenames):
        # 检查文件是否为 PNG 图像
        if filename.endswith('.png'):
            for l in range(14,18):
                if filename.startwith(l):
                    dir_copy = os.path.join(dir_output, dir_name, str(l))
                    if not os.path.exists(dir_copy):
                        os.makedirs(dir_copy)
                    shutil.copy(os.path.join(dir_image, filename), os.path.join(dir_copy, filename))

In [11]:
import pandas as pd
import shutil
import math

from tile_name_calc import get_filename_parent_tile_scale

SPLIT = 'train'
SCALE = 4
LIST_DIR = 'data/sets/list'
DATASET_DIR = 'data/sets/TMGN_1814_source'
DATA_LIST = f'{LIST_DIR}/{SPLIT}_file_data_1816seq.csv'
INPUT_DIR = f'{DATASET_DIR}/{SPLIT}/map_256_source'
COPY_DIR = f'{DATASET_DIR}/{SPLIT}/map_256'
OUTPUT_DIR = f'{DATASET_DIR}/{SPLIT}/ref_{SCALE}_256'
exist_or_create(COPY_DIR)
exist_or_create(OUTPUT_DIR)

zoom_scale = int(math.sqrt(SCALE))
df_filenames_parent = pd.read_csv(DATA_LIST)
print(zoom_scale, df_filenames_parent.shape)

2 (135000, 3)


In [12]:
# 遍历每一行，找出需要的文件
for _, row in tqdm(df_filenames_parent.iterrows(), total=df_filenames_parent.shape[0], desc='Processing'):
    filename = row['filename']
    parent_filename = row['parent_filename']
    position = row['position']

    # 检查文件是否存在
    if os.path.exists(os.path.join(INPUT_DIR, filename)):
        shutil.copyfile(os.path.join(INPUT_DIR, filename), os.path.join(COPY_DIR, filename))
        parent_filename, x, y, size = get_filename_parent_tile_scale(filename, zoom_scale=zoom_scale)

        if not os.path.exists(os.path.join(INPUT_DIR, parent_filename)):
            print(f'Parent file {parent_filename} not found')
            continue

        # 读取图像
        parent_img = cv2.imread(os.path.join(INPUT_DIR, parent_filename), cv2.IMREAD_UNCHANGED)
        cropped_img = parent_img[y:y + size, x:x + size]
        resized_img = cv2.resize(cropped_img, (parent_img.shape[0], parent_img.shape[1]), interpolation=cv2.INTER_AREA)

        # 保存图像
        cv2.imwrite(os.path.join(OUTPUT_DIR, f'{filename}'), resized_img)

Processing:   0%|          | 0/135000 [00:00<?, ?it/s]

In [5]:
import pandas as pd
import math

from tile_name_calc import get_filename_parent_tile_scale

SPLIT = 'train'
SCALE = 4
LIST_DIR = 'data/list'
DATASET_DIR = 'data/sets/TMGN_1814_SOURCE'
DATA_LIST = f'{LIST_DIR}/{SPLIT}_file_data_scale-{SCALE}-256.csv'
SOURCE_DIR = f'{DATASET_DIR}/{SPLIT}/map_256_source'
INPUT_DIR = f'{DATASET_DIR}/{SPLIT}/map_256'
OUTPUT_DIR = f'{DATASET_DIR}/{SPLIT}/ref_{SCALE}_256'

exist_or_create(OUTPUT_DIR)

In [6]:
import cv2
import os

def get_crop_coordinates(x_start, y_start, img_size, position):
    # 根据 position 计算裁剪区域的左上角坐标
    if position == 'top_left':
        x, y = x_start, y_start
    elif position == 'top_right':
        x, y = x_start + img_size // 2, y_start
    elif position == 'bottom_left':
        x, y = x_start, y_start + img_size // 2
    elif position == 'bottom_right':
        x, y = x_start + img_size // 2, y_start + img_size // 2

    return x, y, img_size // 2

def process_batch(batch):
    source = batch.iloc[0]['filename']
    target = batch.iloc[-1]['parent_filename']
    positions = batch['position'].tolist()

    # 检查文件是否存在
    if os.path.exists(os.path.join(INPUT_DIR, source)) and os.path.exists(os.path.join(SOURCE_DIR, target)):
        # 读取图像
        img = cv2.imread(os.path.join(SOURCE_DIR, target), cv2.IMREAD_UNCHANGED)

        x, y = 0, 0
        img_size = img.shape[0]  # Assuming the image is square

        positions.reverse()  # Reverse the list to crop the image from the bottom right corner

        # 计算所有的裁剪坐标
        for position in positions:
            x, y, img_size = get_crop_coordinates(x, y, img_size, position)
            #print(x, y, img_size)

        # 使用最终的裁剪坐标裁剪图像
        cropped_img = img[y:y+img_size, x:x+img_size]

        # 放大图像
        resized_img = cv2.resize(cropped_img, (img.shape[0], img.shape[1]), interpolation = cv2.INTER_AREA)

        # 保存图像
        cv2.imwrite(os.path.join(OUTPUT_DIR, f'{source}'), resized_img)


In [13]:
df_filenames_parent = pd.read_csv(DATA_LIST)
zoom_scale = int(math.sqrt(SCALE))
# 初始化一个空的批次
batch_list = []

# 遍历每一行，找出需要的文件
for index in df_filenames_parent.index:
    # 将行添加到批次中
    row = df_filenames_parent.loc[index]
    batch_list.append(row)

    # 如果批次的大小达到了 zoom_scale，就处理批次
    if len(batch_list) == zoom_scale:
        batch = pd.DataFrame(batch_list, columns=df_filenames_parent.columns)
        process_batch(batch)
        # 清空批次，以便开始下一个批次
        batch_list = []


In [7]:
from tile_name_calc import get_filename_parent_tiles, get_filename_parent_tiles_scale

for filename in tqdm(os.listdir(INPUT_DIR)):
    if filename.endswith('.png'):
        parent_filenames, positions = get_filename_parent_tiles_scale(filename, SCALE)
        current_filename = filename
        parent_list = []
        for parent_filename, position in zip(parent_filenames, positions):
            parent_list.append([current_filename, parent_filename, position])
            current_filename = parent_filename
        parent_batch = pd.DataFrame(parent_list, columns=['filename', 'parent_filename', 'position'])
        process_batch(parent_batch)


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'data/sets/TMGN_1814_SOURCE/train/map_256'

In [7]:
import cv2

def convert_jpg_to_png(jpg_filepath, png_filepath):
    # 读取 JPG 图像
    img = cv2.imread(jpg_filepath)

    # 保存为 PNG 格式
    cv2.imwrite(png_filepath, img)

In [21]:
import os
import shutil
from tqdm.notebook import tqdm

DATASET_DIR = '/mnt/runtime/scx/sgdm-data/datasets/MLMG/CN_dataset/'

path_source = os.path.join(DATASET_DIR, "testA")
path_target = os.path.join(DATASET_DIR, "sgdm", "rs_256")
path_csv = os.path.join(DATASET_DIR, "sgdm", "tilelist_18_15.csv")

os.makedirs(path_target, exist_ok=True)

tilenames_list = []

# 遍历原始路径下的所有文件
for dirpath, dirnames, filenames in os.walk(path_source):
    for filename in tqdm(filenames):
        # 检查文件是否为.png文件
        # if filename.endswith(".png"):
        #     # 获取文件的完整原始路径
        #     full_path = os.path.join(dirpath, filename)

        #     # 生成新的文件名：将路径中的斜杠替换为下划线
        #     relative_path = os.path.relpath(full_path, path_source)  # 获取相对于原始路径的相对路径
        #     new_filename = relative_path.replace(os.sep, '_')
        #     tilenames_list.append(os.path.splitext(new_filename)[0])

        #     # 获取文件的新路径
        #     new_full_path = os.path.join(path_target, new_filename)

        #     # 复制文件到新路径
        #     shutil.copy(full_path, new_full_path)

        if filename.endswith(".jpg"):
            # 获取文件的完整原始路径
            full_path = os.path.join(dirpath, filename)
            tilenames_list.append(os.path.splitext(filename)[0])
            new_filename = filename.replace(".jpg", ".png")
            new_full_path = os.path.join(path_target, new_filename)

            convert_jpg_to_png(full_path, new_full_path)

        elif filename.endswith(".png"):
            tilenames_list.append(os.path.splitext(filename)[0])
            full_path = os.path.join(dirpath, filename)
            new_full_path = os.path.join(path_target, filename)

            shutil.copy(full_path, new_full_path)

0it [00:00, ?it/s]

  0%|          | 0/1700 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/1280 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/320 [00:00<?, ?it/s]

In [20]:
import pandas as pd

# 创建一个DataFrame
df = pd.DataFrame(tilenames_list, columns=['tilename'])

# 对DataFrame进行排序
df = df.sort_values(by='tilename')

# 确保CSV文件所在的目录存在
os.makedirs(os.path.dirname(path_csv), exist_ok=True)

# 将DataFrame导出为CSV文件
df.to_csv(path_csv, index=False)

In [3]:
import os
import pandas as pd
from tqdm.notebook import tqdm

DATASET_DIR = '/home/scx/project/GEN/Generate/SGDM/data/datasets/MLMG/CN_dataset/sgdm/'

path_source = os.path.join(DATASET_DIR, "map_256")
path_csv = os.path.join(DATASET_DIR, "tilelist_18_15.csv")

df_tilenames = pd.DataFrame(columns=['tilename'])

for filename in tqdm(os.listdir(path_source)):
    if filename.endswith('.png'):
        tilename = filename.split('.')[0]
        df_tilenames.loc[len(df_tilenames)] = [tilename]

df_tilenames.to_csv(path_csv, index=False)

  0%|          | 0/1700 [00:00<?, ?it/s]

PermissionError: [Errno 13] Permission denied: '/home/scx/project/GEN/Generate/SGDM/data/datasets/MLMG/CN_dataset/sgdm/tilelist_18_15.csv'

In [6]:
def get_sub_tiles(xtile, ytile, zoom):
    return [(xtile * 2, ytile * 2, zoom + 1),
            (xtile * 2 + 1, ytile * 2, zoom + 1),
            (xtile * 2, ytile * 2 + 1, zoom + 1),
            (xtile * 2 + 1, ytile * 2 + 1, zoom + 1)]

def get_filename_sub_tile(filename):
    # 解析文件名
    parts = filename.split('_')
    z = int(parts[0])
    x = int(parts[1])
    y = int(parts[2].split('.')[0])  # 注意这里我们需要去除文件扩展名

    # 获取下级瓦片
    subtiles = get_sub_tiles(x, y, z)

    # 将下级瓦片转换为文件名格式
    subtile_filenames = [f"{z}_{x}_{y}.png" for (x, y, z) in subtiles]

    return subtile_filenames

def get_filename_sub_tiles(filename, target_zoom=18):
    # 解析文件名
    parts = filename.split('_')
    z = int(parts[0])

    # 创建一个列表来保存结果，并将当前瓦片添加到结果中
    result = [filename]

    # 如果当前缩放级别还没有达到目标缩放级别，获取下级瓦片并递归调用此函数
    if z < target_zoom:
        subtile_filenames = get_filename_sub_tile(filename)
        for subtile_filename in subtile_filenames:
            result.extend(get_filename_sub_tiles(subtile_filename, target_zoom))

    return result

In [12]:
import os
import shutil

FILE_DIR = '/home/scx/project/GEN/Generate/SGDM/data/test/refmap-level-test-range-s2'
INPUT_DIR = f'{FILE_DIR}/target'
OUTPUT_DIR = f'{FILE_DIR}/recursive/14_8189_5447/target'
FILENAME = '14_8189_5447.png'

exist_or_create(OUTPUT_DIR)

tilenames_list = get_filename_sub_tiles(FILENAME, 18)
tilenames_list.remove(FILENAME)
for tilename in tilenames_list:
    assert os.path.exists(os.path.join(INPUT_DIR, tilename)), f"File {tilename} not found"
    shutil.copy(os.path.join(INPUT_DIR, tilename), os.path.join(OUTPUT_DIR, tilename))

In [15]:
tilenames_list = get_filename_sub_tiles(FILENAME, 18)

print(tilenames_list)

def find_min_max(filenames, z_value):
    x_values = []
    y_values = []
    for filename in filenames:
        parts = filename.split('_')
        z = int(parts[0])
        x = int(parts[1])
        y = int(parts[2].split('.')[0])  # 注意这里我们需要去除文件扩展名
        if z == z_value:
            x_values.append(x)
            y_values.append(y)
    return min(x_values), max(x_values), min(y_values), max(y_values)

# 假设你的文件名列表是 filenames
min_x, max_x, min_y, max_y = find_min_max(tilenames_list, 18)

print(f"Min x: {min_x}, Max x: {max_x}, Min y: {min_y}, Max y: {max_y}")


['14_8189_5447.png', '15_16378_10894.png', '16_32756_21788.png', '17_65512_43576.png', '18_131024_87152.png', '18_131025_87152.png', '18_131024_87153.png', '18_131025_87153.png', '17_65513_43576.png', '18_131026_87152.png', '18_131027_87152.png', '18_131026_87153.png', '18_131027_87153.png', '17_65512_43577.png', '18_131024_87154.png', '18_131025_87154.png', '18_131024_87155.png', '18_131025_87155.png', '17_65513_43577.png', '18_131026_87154.png', '18_131027_87154.png', '18_131026_87155.png', '18_131027_87155.png', '16_32757_21788.png', '17_65514_43576.png', '18_131028_87152.png', '18_131029_87152.png', '18_131028_87153.png', '18_131029_87153.png', '17_65515_43576.png', '18_131030_87152.png', '18_131031_87152.png', '18_131030_87153.png', '18_131031_87153.png', '17_65514_43577.png', '18_131028_87154.png', '18_131029_87154.png', '18_131028_87155.png', '18_131029_87155.png', '17_65515_43577.png', '18_131030_87154.png', '18_131031_87154.png', '18_131030_87155.png', '18_131031_87155.png', '